In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [93]:
%load_ext autoreload
%autoreload 2
from trader.tradertk import *
from trader.utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
with open('data/retDAX.txt', 'r') as f:
    tmp = f.read()
    
retDAX = list(map(lambda x: float(x), tmp.split()))

with open('data/DAX.txt', 'r') as f:
    tmp = f.read()
    
DAX = list(map(lambda x: float(x), tmp.split()))

M = 10  # The number of time series inputs to the trader.
T = 500
N = 100

initial_theta = np.ones((M+2, 1))  # Initialize theta.

X = retDAX

Xn = featureNormalize(X)

# Ft = np.zeros((T+1, 1))
Ft = np.ones(T+1)  # Holdings at period t.

miu = 1  # maximum possible number of shares per transaction
delta = 0.001  # The cost for a transaction at period t

Ret, sharp = rewardFunction(X, miu, delta, Ft, M)

In [99]:
# def costF(t):
#     return costFunction(Xn[1:M+T], X[1:M+T], t)

# from scipy.optimize import minimize
# minimize(costF, initial_theta)

In [90]:
# initial_theta = np.ones((M+2, 1))

tmp = costFunction(X, Xn, np.random.uniform(-1,1,(M+2, 1)))
tmp

-0.000448662391019
[[ 8.25431566  8.12964843  8.43933623 ...,  8.25920124  8.28989735
   7.93749305]
 [ 8.25431566  8.12964843  8.43933623 ...,  8.25920124  8.28989735
   7.93749305]
 [ 8.25431566  8.12964843  8.43933623 ...,  8.25920124  8.28989735
   7.93749305]
 ..., 
 [ 8.25431566  8.12964843  8.43933623 ...,  8.25920124  8.28989735
   7.93749305]
 [ 8.25431566  8.12964843  8.43933623 ...,  8.25920124  8.28989735
   7.93749305]
 [ 8.25431566  8.12964843  8.43933623 ...,  8.25920124  8.28989735
   7.93749305]]
(10, 5415)
(64980, 10)
(12, 12)
(12, 12)


ValueError: shapes (64980,10) and (12,12) not aligned: 10 (dim 1) != 12 (dim 0)

In [55]:
np.tile([1,2,3], (M,1))

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])